# Preprocessing of raw blood donation data for use in SVMs - Finnish data

### Importing packages

In [1]:
import numpy as np
import pandas as pd 
import datetime
import ast
import pickle
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

### Loading original data files 
'vdonationc' contains all donation visits but information on genetic markers is present only in 'lastdon' so we will merge them.

In [ ]:
data_last = pd.read_csv('../../data/lastdon.csv', low_memory=False)
data_all = pd.read_csv('../../data/vdonationc.csv', low_memory=False)
data_raw = data_all.merge(data_last[['vdonor', 'height', 'weight', 'smoking', 'bmi', 
                                     'snp_17_58358769', 'snp_6_32617727', 'snp_15_45095352',
                                     'snp_1_169549811', 'prs_anemia', 'prs_ferritin', 'prs_hemoglobin']], 
                          how='inner',
                          on='vdonor')
print(data_all.shape, data_raw.shape)

In [ ]:
data_raw.columns

Out of 5 786 320 total donations in vdonationc, we have genetic markers for 645 978. 

### Selecting variables and relevant visits
Drop some columns that are not needed: zip code, city, donation site and blood group. Blood group might be interesting to include, but factors have to be dummy-coded for SVMs. For now, drop it. Only keep successful whole-blood donations (donat_phleb == 'K') and Hb-deferred ones (Hb_deferral == 1 & donat_phleb == '\*') from the past 5 years (2016-2020). 

In [ ]:
data = data_raw.copy().drop(columns=['Unnamed: 0', 'zip', 'city', 'site', 'aborh'])
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data = data.loc[(data.donat_phleb == 'K') | ((data.donat_phleb == '*') & (data.Hb_deferral == 1)), ]
print(data_raw.shape, data.shape)

data = data.drop(columns=['status','donat_phleb'])
#data.to_pickle('../../data/alldata.pkl')

data = data.loc[data.year > 2015, ]
#data.to_pickle('../../data/alldata_2016_2020.pkl')
print(data.shape)

This leaves us with 201 843 donation attempts.

Add variables HbPrevn and DaystoPrevn for n between 1-5. 

In [ ]:
def add_prev_hb_time(df, number):
    colnames = ['HbPrev'+str(number), 'DaystoPrev'+str(number)]
    df[colnames[0]] = df['Hb'].shift(number)
    df[colnames[1]] = (df['date'] - df['date'].shift(number)) / pd.Timedelta('1 day') 
    return df

data = data.dropna(axis=0)

df_1 = data.groupby('vdonor').apply(add_prev_hb_time, number=1)
print(datetime.datetime.now(), '1 done')
df_2 = df_1.groupby('vdonor').apply(add_prev_hb_time, number=2)
print(datetime.datetime.now(), '2 done')
df_3 = df_2.groupby('vdonor').apply(add_prev_hb_time, number=3)
print(datetime.datetime.now(), '3 done')
df_4 = df_3.groupby('vdonor').apply(add_prev_hb_time, number=4)
print(datetime.datetime.now(), '4 done')
df_5 = df_4.groupby('vdonor').apply(add_prev_hb_time, number=5)
print(datetime.datetime.now(), '5 done')
df_5.head()

df_5.to_pickle('../../data/df_2016_2020.pkl')

### Checking pre-processed data

In [ ]:
df = pd.read_pickle('../../data/df_2016_2020.pkl')
df.describe()

Min, max and mean for most variables look sensible. However the minimum value for several DaystoPrevn variables is very low, so we investigate this. From the Veripalvelu website: *The minimum interval between blood donations is 91 days for women and 61 days for men.* Values smaller than this can occur if the previous visit was a deferral, however. This is the case for almost all these observations. There are a few (<100) observations that have DaystoPrevn variables smaller than the minimum donation interval, but they do appear to be valid donations. We leave them in the dataset.

A look at the distribution of number of donation attempts per donor:

In [ ]:
dcounts = df.vdonor.value_counts()

plt.hist(dcounts)
plt.show()

# Marginal distributions of variables per SVM

We want to describe the distributions of all variables in the data set separately for women and men, and separately for the five models SVM-1 through SVM-5. 

In [79]:
df = pd.read_pickle('../../data/df_2016_2020.pkl')
df['prs_anemia'] = df['prs_anemia'].round(10)
df['prs_ferritin'] = df['prs_ferritin'].round(10)
df['prs_hemoglobin'] = df['prs_hemoglobin'].round(10)

In [80]:
df.head()

,vdonor,age,sex,date,Hb,Hb_deferral,height,weight,smoking,bmi,snp_17_58358769,snp_6_32617727,snp_15_45095352,snp_1_169549811,prs_anemia,prs_ferritin,prs_hemoglobin,year,month,HbPrev1,DaystoPrev1,HbPrev2,DaystoPrev2,HbPrev3,DaystoPrev3,HbPrev4,DaystoPrev4,HbPrev5,DaystoPrev5
49,MV4ALL7LH8X3,52,Women,2016-03-31,131.0,0,163,79,False,29.7339,0,1,0,0,0.000001,-1.510000e-08,-0.000002,2016,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,MV4ALL7LH8X3,52,Women,2016-07-05,118.0,1,163,79,False,29.7339,0,1,0,0,0.000001,-1.510000e-08,-0.000002,2016,7,131.0,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,MV4ALL7LH8X3,52,Women,2016-10-06,125.0,0,163,79,False,29.7339,0,1,0,0,0.000001,-1.510000e-08,-0.000002,2016,10,118.0,93.0,131.0,189.0,NaN,NaN,NaN,NaN,NaN,NaN
52,MV4ALL7LH8X3,53,Women,2016-12-30,121.0,1,163,79,False,29.7339,0,1,0,0,0.000001,-1.510000e-08,-0.000002,2016,12,125.0,85.0,118.0,178.0,131.0,274.0,NaN,NaN,NaN,NaN
53,MV4ALL7LH8X3,53,Women,2017-03-23,138.0,0,163,79,False,29.7339,0,1,0,0,0.000001,-1.510000e-08,-0.000002,2017,3,121.0,83.0,125.0,168.0,118.0,261.0,131.0,357.0,NaN,NaN


In [77]:
def describe_numerical(x):
    xmin, xq1, xmedian, xq3, xmax = np.percentile(x, [0, 25, 50, 75, 100]) 
    return(xmin, xq1, xmedian, xq3, xmax)

def describe_categorical(x):
    xoccs = x.value_counts(sort=False)
    xoccsn = x.value_counts(normalize=True, sort=False)
#    df = pd.DataFrame({'Value':xoccs.index,
#                       'Count':xoccs.values,
#                       'Proportion': xoccsn.values})
    return(xoccs.index, xoccs.values, xoccsn.values)

def describe_df(df):
    xcount = df.shape[1]
    # numerical variables
    df_num = df.drop(columns=['year', 'smoking', 'snp_17_58358769', 'snp_6_32617727', 'snp_15_45095352', 'snp_1_169549811'])
    res_num = df_num.apply(describe_numerical, axis='index').transpose()
    res_num.columns = ['minimum', 'Q1', 'median', 'Q3', 'maximum']
    # categorical variables
    res_cat = []
    for variable in ['year', 'smoking', 'snp_17_58358769', 'snp_6_32617727', 'snp_15_45095352', 'snp_1_169549811']:
        res = describe_categorical(df[variable])
        res = pd.DataFrame(res).transpose()
        res.columns = ['Value','Count','Proportion']
        res_cat.append(res)
    return(res_num, res_cat)

### Male donors

In [78]:
df_m = df.loc[df['sex'] == 'Men', ['year', 'age', 'month', 'smoking', 'height', 'weight', 'bmi',
                         'snp_17_58358769', 'snp_6_32617727', 'snp_15_45095352', 'snp_1_169549811', 'prs_anemia', 'prs_ferritin',
                         'prs_hemoglobin', 'HbPrev1', 'DaystoPrev1', 'HbPrev2', 'DaystoPrev2','HbPrev3', 'DaystoPrev3',
                         'HbPrev4', 'DaystoPrev4', 'HbPrev5', 'DaystoPrev5']]

for nback in range(1, 6):
    varname = 'HbPrev' + str(nback)
    df_sub = df_m.dropna(subset=['HbPrev'+str(nback)])
    print(nback, 'donations back')
    res_num, res_cat = describe_df(df_sub)
    print('Numerical variables:')
    display(res_num)
    for i, varname in enumerate(['year', 'smoking', 'snp_17_58358769', 'snp_6_32617727', 'snp_15_45095352', 'snp_1_169549811']):
        print('Variable', varname)
        display(res_cat[i])

1 donations back
Numerical variables:


,minimum,Q1,median,Q3,maximum
age,18.000000,3.800000e+01,5.200000e+01,6.000000e+01,71.000000
month,1.000000,3.000000e+00,6.000000e+00,1.000000e+01,12.000000
height,149.000000,1.750000e+02,1.800000e+02,1.840000e+02,210.000000
weight,50.000000,7.800000e+01,8.600000e+01,9.600000e+01,190.000000
bmi,15.319690,2.456931e+01,2.670362e+01,2.942508e+01,65.192744
prs_anemia,-0.000004,-8.773000e-07,-3.880000e-08,7.920000e-07,0.000005
prs_ferritin,-0.000007,-1.271712e-06,-2.347990e-08,1.241847e-06,0.000008
prs_hemoglobin,-0.000017,-3.095667e-06,1.816460e-08,3.254924e-06,0.000016
HbPrev1,102.000000,1.470000e+02,1.540000e+02,1.610000e+02,196.000000
DaystoPrev1,1.000000,7.100000e+01,9.800000e+01,1.470000e+02,1212.000000


Variable year


,Value,Count,Proportion
0,2017.0,21036.0,0.235497
1,2018.0,24767.0,0.277265
2,2019.0,24552.0,0.274858
3,2016.0,12425.0,0.139097
4,2020.0,6546.0,0.073282


Variable smoking


...

,Value,Count,Proportion
0,2.0,10455.0,0.206780
1,1.0,24855.0,0.491584
2,0.0,15251.0,0.301636


Variable snp_15_45095352


,Value,Count,Proportion
0,0.0,44723.0,0.884536
1,1.0,5556.0,0.109887
2,2.0,282.0,0.005577


Variable snp_1_169549811


,Value,Count,Proportion
0,1.0,2016.0,0.039873
1,0.0,48526.0,0.959752
2,2.0,19.0,0.000376


### Female donors

In [60]:
df_f = df.loc[df['sex'] == 'Women', ['year', 'age', 'month', 'smoking', 'height', 'weight', 'bmi',
                         'snp_17_58358769', 'snp_6_32617727', 'snp_15_45095352', 'snp_1_169549811', 'prs_anemia', 'prs_ferritin',
                         'prs_hemoglobin', 'HbPrev1', 'DaystoPrev1', 'HbPrev2', 'DaystoPrev2','HbPrev3', 'DaystoPrev3',
                         'HbPrev4', 'DaystoPrev4', 'HbPrev5', 'DaystoPrev5']]

for nback in range(1, 6):
    varname = 'HbPrev' + str(nback)
    df_sub = df_f.dropna(subset=['HbPrev'+str(nback)])
    print(nback, 'donations back')
    res_num, res_cat = describe_df(df_sub)
    print('Numerical variables:')
    display(res_num)
    for i, varname in enumerate(['year', 'smoking', 'snp_17_58358769', 'snp_6_32617727', 'snp_15_45095352', 'snp_1_169549811']):
        print('Variable', varname)
        display(res_cat[i])

1 donations back
Numerical variables:


,minimum,Q1,median,Q3,maximum
age,18.000000,2.900000e+01,4.600000e+01,5.700000e+01,70.000000
month,1.000000,3.000000e+00,6.000000e+00,1.000000e+01,12.000000
height,138.000000,1.620000e+02,1.660000e+02,1.700000e+02,198.000000
weight,49.000000,6.400000e+01,7.100000e+01,8.200000e+01,172.000000
bmi,15.942183,2.318367e+01,2.578125e+01,2.939469e+01,64.776711
prs_anemia,-0.000005,-8.472577e-07,-2.266920e-09,8.279037e-07,0.000005
prs_ferritin,-0.000007,-1.193138e-06,3.224310e-08,1.279313e-06,0.000008
prs_hemoglobin,-0.000016,-3.010211e-06,3.850150e-08,3.105051e-06,0.000018
HbPrev1,86.000000,1.330000e+02,1.400000e+02,1.470000e+02,187.000000
DaystoPrev1,2.000000,1.040000e+02,1.320000e+02,2.060000e+02,1421.000000


Variable year


,Value,Count,Proportion
0,2016.0,9249.0,0.109488
1,2017.0,19230.0,0.227641
2,2018.0,24973.0,0.295626
3,2019.0,24768.0,0.293199
4,2020.0,6255.0,0.074046


Variable smoking


...

,Value,Count,Proportion
0,1.0,16301.0,0.491024
1,0.0,10576.0,0.318573
2,2.0,6321.0,0.190403


Variable snp_15_45095352


,Value,Count,Proportion
0,0.0,29065.0,0.875505
1,1.0,3994.0,0.120308
2,2.0,139.0,0.004187


Variable snp_1_169549811


,Value,Count,Proportion
0,0.0,31739.0,0.956052
1,1.0,1425.0,0.042924
2,2.0,34.0,0.001024


# Scaled based on training data

We need to scale all explanatory variables before doing anything with the SVM. We use the StandardScaler option in the sk-learn package, which makes all variables have a mean of zero and variance of one. We save the scalers for later use when we change time-related variables. Scalers are fitted using only the training data and then used to transform both training and test data.

Test data will be the last year of donations (1 May 2019 - 31 April 2020) and training data everything before that.

In [ ]:
df = pd.read_pickle('../../data/df_2016_2020.pkl')

In [ ]:
var = ['vdonor', 'date', 'sex', 'year', 'age', 'month', 'smoking', 'height', 'weight', 'bmi', 
       'snp_17_58358769', 'snp_6_32617727', 'snp_15_45095352', 'snp_1_169549811', 'prs_anemia', 'prs_ferritin',
       'prs_hemoglobin']
for n in range(1, 6):
    var.extend(['HbPrev'+str(n), 'DaystoPrev'+str(n)])
var.append('Hb_deferral')

df['smoking'] = df['smoking'].astype(int)

train_men = df.loc[(df.sex == 'Men') & (df.date <= '2019-05-01'), var]
train_men = train_men[train_men.columns[4:]]
train_women = df.loc[(df.sex == 'Women') & (df.date <= '2019-05-01'), var]
train_women = train_women[train_women.columns[4:]]

test_men = df.loc[(df.sex == 'Men') & (df.date > '2019-05-01'), var]
test_men = test_men[test_men.columns[4:]]
test_women = df.loc[(df.sex == 'Women') & (df.date > '2019-05-01'), var]
test_women = test_women[test_women.columns[4:]]

In [ ]:
for nback in range(1, 6):
    var = ['age', 'month', 'smoking', 'height', 'weight', 'bmi', 
           'snp_17_58358769', 'snp_6_32617727', 'snp_15_45095352', 'snp_1_169549811', 'prs_anemia', 'prs_ferritin',
           'prs_hemoglobin']
    for n in range(1, nback+1):
        var.extend(['HbPrev'+str(n), 'DaystoPrev'+str(n)])
    var.append('Hb_deferral')
    
    train_men_sub = train_men[var].dropna()
    train_women_sub = train_women[var].dropna()
    test_men_sub = test_men[var].dropna()
    test_women_sub = test_women[var].dropna()
    
    scaler_men = StandardScaler()
    scaler_women = StandardScaler()
    scaler_men.fit(train_men_sub[train_men_sub.columns[:-1]])
    scaler_women.fit(train_women_sub[train_men_sub.columns[:-1]])
    
    train_men_sub[train_men_sub.columns[:-1]] = scaler_men.transform(train_men_sub[train_men_sub.columns[:-1]])
    train_women_sub[train_women_sub.columns[:-1]] = scaler_women.transform(train_women_sub[train_women_sub.columns[:-1]])
    test_men_sub[test_men_sub.columns[:-1]] = scaler_men.transform(test_men_sub[test_men_sub.columns[:-1]])
    test_women_sub[test_women_sub.columns[:-1]] = scaler_women.transform(test_women_sub[test_women_sub.columns[:-1]])
    
    pickle.dump(scaler_men, open('../results/scalers/men_'+str(nback)+'.pkl', 'wb'))
    pickle.dump(scaler_women, open('../results/scalers/women_'+str(nback)+'.pkl', 'wb'))
    
    train_men_sub.to_pickle('../../data/scaled/men_'+str(nback)+'_train.pkl')
    train_women_sub.to_pickle('../../data/scaled/women_'+str(nback)+'_train.pkl')
    test_men_sub.to_pickle('../../data/scaled/men_'+str(nback)+'_test.pkl')
    test_women_sub.to_pickle('../../data/scaled/women_'+str(nback)+'_test.pkl')

# Limited variables ('onlyhb'), scaled on training data
To draw comparisons with results on the Dutch data, we also fit the model on data only containing the variables present in the Dutch data. This has the same result as removing those variables from the scaled data created above.

In [ ]:
df = pd.read_pickle('../../data/df_2016_2020.pkl')

In [ ]:
var = ['vdonor', 'date', 'sex', 'year', 'age', 'month']
for n in range(1, 6):
    var.extend(['HbPrev'+str(n), 'DaystoPrev'+str(n)])
var.append('Hb_deferral')

train_men = df.loc[(df.sex == 'Men') & (df.date <= '2019-05-01'), var]
train_men = train_men[train_men.columns[4:]]
train_women = df.loc[(df.sex == 'Women') & (df.date <= '2019-05-01'), var]
train_women = train_women[train_women.columns[4:]]

test_men = df.loc[(df.sex == 'Men') & (df.date > '2019-05-01'), var]
test_men = test_men[test_men.columns[4:]]
test_women = df.loc[(df.sex == 'Women') & (df.date > '2019-05-01'), var]
test_women = test_women[test_women.columns[4:]]

In [ ]:
for nback in range(1, 6):
    var = ['age', 'month']
    for n in range(1, nback+1):
        var.extend(['HbPrev'+str(n), 'DaystoPrev'+str(n)])
    var.append('Hb_deferral')
    
    train_men_sub = train_men[var].dropna()
    train_women_sub = train_women[var].dropna()
    test_men_sub = test_men[var].dropna()
    test_women_sub = test_women[var].dropna()
    
    scaler_men = StandardScaler()
    scaler_women = StandardScaler()
    scaler_men.fit(train_men_sub[train_men_sub.columns[:-1]])
    scaler_women.fit(train_women_sub[train_men_sub.columns[:-1]])
    
    pickle.dump(scaler_men, open('../results/scalers_onlyhb/men_'+str(nback)+'.pkl', 'wb'))
    pickle.dump(scaler_women, open('../results/scalers_onlyhb/women_'+str(nback)+'.pkl', 'wb'))
    
    train_men_sub[train_men_sub.columns[:-1]] = scaler_men.transform(train_men_sub[train_men_sub.columns[:-1]])
    train_women_sub[train_women_sub.columns[:-1]] = scaler_women.transform(train_women_sub[train_women_sub.columns[:-1]])
    test_men_sub[test_men_sub.columns[:-1]] = scaler_men.transform(test_men_sub[test_men_sub.columns[:-1]])
    test_women_sub[test_women_sub.columns[:-1]] = scaler_women.transform(test_women_sub[test_women_sub.columns[:-1]])
    
    train_men_sub.to_pickle('../../data/scaled_onlyhb/men_'+str(nback)+'_train.pkl')
    train_women_sub.to_pickle('../../data/scaled_onlyhb/women_'+str(nback)+'_train.pkl')
    test_men_sub.to_pickle('../../data/scaled_onlyhb/men_'+str(nback)+'_test.pkl')
    test_women_sub.to_pickle('../../data/scaled_onlyhb/women_'+str(nback)+'_test.pkl')

# Another run with only Hb + genetic data
This excludes weight, height, bmi and smoking. This allows us to see the relative added value of genetic data only.

In [3]:
df = pd.read_pickle('../../data/df_2016_2020.pkl')
df.head()

,vdonor,age,sex,date,Hb,Hb_deferral,height,weight,smoking,bmi,snp_17_58358769,snp_6_32617727,snp_15_45095352,snp_1_169549811,prs_anemia,prs_ferritin,prs_hemoglobin,year,month,HbPrev1,DaystoPrev1,HbPrev2,DaystoPrev2,HbPrev3,DaystoPrev3,HbPrev4,DaystoPrev4,HbPrev5,DaystoPrev5
49,MV4ALL7LH8X3,52,Women,2016-03-31,131.0,0,163,79,False,29.7339,0,1,0,0,0.000001,-1.510828e-08,-0.000002,2016,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,MV4ALL7LH8X3,52,Women,2016-07-05,118.0,1,163,79,False,29.7339,0,1,0,0,0.000001,-1.510828e-08,-0.000002,2016,7,131.0,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,MV4ALL7LH8X3,52,Women,2016-10-06,125.0,0,163,79,False,29.7339,0,1,0,0,0.000001,-1.510828e-08,-0.000002,2016,10,118.0,93.0,131.0,189.0,NaN,NaN,NaN,NaN,NaN,NaN
52,MV4ALL7LH8X3,53,Women,2016-12-30,121.0,1,163,79,False,29.7339,0,1,0,0,0.000001,-1.510828e-08,-0.000002,2016,12,125.0,85.0,118.0,178.0,131.0,274.0,NaN,NaN,NaN,NaN
53,MV4ALL7LH8X3,53,Women,2017-03-23,138.0,0,163,79,False,29.7339,0,1,0,0,0.000001,-1.510828e-08,-0.000002,2017,3,121.0,83.0,125.0,168.0,118.0,261.0,131.0,357.0,NaN,NaN


In [4]:
var = ['vdonor', 'date', 'sex', 'year', 'age', 'month', 'snp_17_58358769', 'snp_6_32617727',  
       'snp_15_45095352', 'snp_1_169549811', 'prs_anemia', 'prs_ferritin', 'prs_hemoglobin']
for n in range(1, 6):
    var.extend(['HbPrev'+str(n), 'DaystoPrev'+str(n)])
var.append('Hb_deferral')

train_men = df.loc[(df.sex == 'Men') & (df.date <= '2019-05-01'), var]
train_men = train_men[train_men.columns[4:]]
train_women = df.loc[(df.sex == 'Women') & (df.date <= '2019-05-01'), var]
train_women = train_women[train_women.columns[4:]]

test_men = df.loc[(df.sex == 'Men') & (df.date > '2019-05-01'), var]
test_men = test_men[test_men.columns[4:]]
test_women = df.loc[(df.sex == 'Women') & (df.date > '2019-05-01'), var]
test_women = test_women[test_women.columns[4:]]

In [6]:
for nback in range(1, 6):
    var = ['age', 'month', 'snp_17_58358769', 'snp_6_32617727', 'snp_15_45095352', 
           'snp_1_169549811', 'prs_anemia', 'prs_ferritin', 'prs_hemoglobin']
    for n in range(1, nback+1):
        var.extend(['HbPrev'+str(n), 'DaystoPrev'+str(n)])
    var.append('Hb_deferral')
    
    train_men_sub = train_men[var].dropna()
    train_women_sub = train_women[var].dropna()
    test_men_sub = test_men[var].dropna()
    test_women_sub = test_women[var].dropna()
    
    scaler_men = StandardScaler()
    scaler_women = StandardScaler()
    scaler_men.fit(train_men_sub[train_men_sub.columns[:-1]])
    scaler_women.fit(train_women_sub[train_men_sub.columns[:-1]])
    
    pickle.dump(scaler_men, open('../results/scalers_hbgen/men_'+str(nback)+'.pkl', 'wb'))
    pickle.dump(scaler_women, open('../results/scalers_hbgen/women_'+str(nback)+'.pkl', 'wb'))
    
    train_men_sub[train_men_sub.columns[:-1]] = scaler_men.transform(train_men_sub[train_men_sub.columns[:-1]])
    train_women_sub[train_women_sub.columns[:-1]] = scaler_women.transform(train_women_sub[train_women_sub.columns[:-1]])
    test_men_sub[test_men_sub.columns[:-1]] = scaler_men.transform(test_men_sub[test_men_sub.columns[:-1]])
    test_women_sub[test_women_sub.columns[:-1]] = scaler_women.transform(test_women_sub[test_women_sub.columns[:-1]])
    
    train_men_sub.to_pickle('../../data/scaled_hbgen/men_'+str(nback)+'_train.pkl')
    train_women_sub.to_pickle('../../data/scaled_hbgen/women_'+str(nback)+'_train.pkl')
    test_men_sub.to_pickle('../../data/scaled_hbgen/men_'+str(nback)+'_test.pkl')
    test_women_sub.to_pickle('../../data/scaled_hbgen/women_'+str(nback)+'_test.pkl')